In [ ]:
from keras import layers
from keras import models
from keras.optimizers import adam
import tensorflow as tf
import cv2

from DQN import Agent

import numpy as np
import gym

In [ ]:
env_name = 'Breakout-v0'
env = gym.make(env_name)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [ ]:
img_height = 84
img_width = 84
img_channels = 4

state_dim = [img_height,img_width,img_channels]

In [ ]:
def preprocess(observation):
    observation = cv2.cvtColor(cv2.resize(observation, (84, 110)), cv2.COLOR_BGR2GRAY)
    observation = observation[26:110,:]
    ret, observation = cv2.threshold(observation,1,255,cv2.THRESH_BINARY)
    return np.reshape(observation,(84,84,1))


In [ ]:
env.reset()
action = np.random.randint(low=0, high=nb_actions)
img, reward, end_episode, info = env.step(action=action)


a = np.array(preprocess(img))
print(a.shape)

In [ ]:
action = np.random.randint(low=0, high=nb_actions)
img, reward, end_episode, info = env.step(action=action)
b = np.array(preprocess(img))

action = np.random.randint(low=0, high=nb_actions)
img, reward, end_episode, info = env.step(action=action)
c = np.array(preprocess(img))

state = np.dstack([a, b,c])
print(state.shape)


In [ ]:
DQN_Agent = Agent(state_dim,nb_actions)

In [ ]:
DQN_Agent.model.summary()